<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/intermediate-results/notebooks/4.0b-mb-gbm_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost==1.1
!pip install gcsfs==2022.10.0
!pip install ipywidgets==8.0.2
!pip install numpy==1.23.4
!pip install pandas==1.5.1
!pip install scikit-learn==1.1.3
!pip install fastparquet
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.7 MB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 12.3 MB/s 
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 6.9 MB/s 
     |████████████████████████████████| 2.0 MB 56.0 MB/s 
     |████████████████████████████████| 1.6 MB 53.5 MB/s 
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.1
    Uninstalling widgetsnbextension-3.6.1:
      Successfully uninstalled widgetsnbextension-3.6.1
  Attempting uninstall: ipywidgets
    Found exist

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.2 MB 8.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.2 MB 1.2 MB/s 
     |████████████████████████████████| 16.9 MB 50.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 8.3 MB/s 
     |████████████████████████████████| 2.0 MB 63.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 6.3 MB/s 
     |████████████████████████████████| 168 kB 62.2 MB/s 
     |████████████████████████████████| 182 kB 44.1 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 69.7 MB/s 
     |████████████████████████████████| 166 kB 77.5 MB/s 
     |████████████████████████████████| 166 kB 50.8 MB/s 
     |████████████████████████████████| 162 kB 81.5 MB/s 
     |████████████████████████████████| 162 kB 70.7 MB/s 
     |████████████████████████████████| 158 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 64.9 MB/s 
     |████████████████████████████████| 157 kB 81.1 

Run `pip install .` first to install all dependencies.

In [2]:
from catboost import CatBoostClassifier

import gcsfs
import google.auth
from google.colab import auth, output

import numpy as np
import pandas as pd

import os
from pathlib import Path

from sklearn.metrics import accuracy_score

from tqdm.notebook import tqdm
import wandb

from typing import List, Optional

In [3]:
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)
fs_prefix = "gs://"

In [4]:
features_classical_size = [
    'TRADE_PRICE', 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',
       'rel_ask_size_ex', 'TRADE_SIZE', 'bid_size_ex', 'ask_size_ex',
       'rel_ask_ex', 'rel_bid_ex', 'BEST_rel_bid', 'BEST_rel_ask',
       'bid_ask_ratio_ex', 'chg_ex_lead', 'chg_ex_lag', 'chg_all_lead',
       'chg_all_lag', 'ask_ex', 'bid_ex', 'BEST_ASK', 'BEST_BID',
       'price_all_lag', 'price_all_lead', 'price_ex_lag', 'price_ex_lead'
]

In [8]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
# for refs

run = wandb.init(project="thesis",entity="fbv")

dataset = "fbv/thesis/classical_size_features_normalized:v0"
artifact = run.use_artifact(dataset)
data_dir = artifact.download()

model = "fbv/thesis/kwlaw02g_CatBoostClassifier_default_trial_42.cbm:v0"
artifact = run.use_artifact(model)
model_dir = artifact.download()

wandb: Downloading large artifact classical_size_features_normalized:v0, 2564.03MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:0.0
wandb:   1 of 1 files downloaded.  


In [13]:
X_test = pd.read_parquet(Path(data_dir, "test_set_20.parquet"), engine="fastparquet")

y_test = X_test["buy_sell"]
X_test = X_test[features_classical_size]


In [14]:
X_test.head()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,TRADE_SIZE,bid_size_ex,ask_size_ex,rel_ask_ex,rel_bid_ex,BEST_rel_bid,...,chg_all_lead,chg_all_lag,ask_ex,bid_ex,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead
39342171,-0.999253,-0.999944,-0.999978,-0.999959,-0.999964,-0.99938,-0.99978,-0.450753,0.450753,-0.222609,...,0.979614,0.887864,-0.999125,-0.999235,-0.999994,-0.999235,-0.999908,-0.999989,-0.999341,-0.999321
39342172,-0.988555,-0.999980,-0.999660,-0.999773,-1.000000,-0.99998,-0.99998,-0.449826,0.449826,-0.221914,...,0.979609,0.887724,-0.988111,-0.988814,-0.999919,-0.988814,-0.998300,-0.999837,-0.987734,-0.994284
39342173,-0.957677,-0.999980,-0.999660,-0.999773,-1.000000,-0.99998,-0.99998,-0.450689,0.450689,-0.222561,...,0.979589,0.887492,-0.953246,-0.957610,-0.999680,-0.957613,-0.993839,-0.999395,-0.957501,-0.965612
39342174,-0.996717,-0.999980,-0.999966,-0.999977,-1.000000,-0.99980,-0.99980,-0.448436,0.448436,-0.220290,...,0.979647,0.887998,-0.996718,-0.997277,-0.999979,-0.997277,-0.999704,-0.999987,-1.000000,-1.000000
39342175,-0.997523,-0.999980,-0.999966,-0.999977,-1.000000,-0.99980,-0.99980,-0.450753,0.450753,-0.222609,...,0.979636,0.887758,-0.997155,-0.997498,-0.999981,-0.997498,-0.999560,-0.999987,-1.000000,-1.000000


## CatBoost Baseline 🐈‍⬛

In [19]:
model = CatBoostClassifier()
model.load_model(fname=Path(model_dir, "kwlaw02g_CatBoostClassifier_default_trial_42.cbm"))

In [20]:
acc = model.score(X_test, y_test)

print(acc)

/usr/local/lib/python3.8/dist-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0.7156786095853239


### Robustness Baseline🥊

In [22]:
test_orig = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet",
    engine="fastparquet",
)

In [23]:
# Copy unscaled columns
X_print = test_orig.copy()
# X_print = pd.concat([train, val, test])

# add baseline results
X_print["rule"] = "Baseline"
X_print["buy_sell_predicted"] = model.predict(X_test)

# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)
X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-np.inf, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

bins_ttm = [-np.inf, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# TODO: Security type
# TODO: Moneyness
# TODO: time from previous trade; same underlying or any?


/usr/local/lib/python3.8/dist-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [24]:
def check_robustness(criterion: str = "year_binned") -> pd.DataFrame:
    """
    Check robustness of rules by calculating the accuracy for a given
    criterion and rules.

    Example:
    rule		Baseline
    TRADE_SIZE_binned
    (0,1]	  0.710966
    (1,3]	  0.717664
    (3,5]	  0.715195
    (5,11]	0.699428
    >11	  	0.688348

    Args:
        criterion (str, optional): criterion to check robustness for.
        Defaults to "year_binned".

    Returns:
        pd.DataFrame: DataFrame with accuracy of rules. Rule in columns and
        criterion values in rows.
    """

    # fill others randomly with equal weight for every class.
    X_print["buy_sell_predicted"] = X_print["buy_sell_predicted"].map(
        lambda l: l if not np.isnan(l) else np.random.choice([-1, 1])
    )

    # cuculate average over columns if multiple subsets are combined
    results = (
        X_print.groupby(["rule", criterion])[["buy_sell", "buy_sell_predicted"]]
        .apply(lambda x: accuracy_score(x["buy_sell"], x["buy_sell_predicted"]))
        .unstack(level=0)
        .assign(avg=lambda x: x.mean(axis=1))
    )
    return results


In [25]:
check_robustness("year_binned")


rule,Baseline,avg
year_binned,,
2015,0.684294,0.684294
2016,0.718206,0.718206
2017,0.720383,0.720383


In [26]:
check_robustness("OPTION_TYPE")


rule,Baseline,avg
OPTION_TYPE,,
C,0.711115,0.711115
P,0.720891,0.720891


In [27]:
check_robustness("TRADE_SIZE_binned")


rule,Baseline,avg
TRADE_SIZE_binned,,
"(0,1]",0.719992,0.719992
"(1,3]",0.723841,0.723841
"(3,5]",0.721618,0.721618
"(5,11]",0.707081,0.707081
>11,0.700010,0.700010


In [28]:
check_robustness("ttm_binned")


rule,Baseline,avg
ttm_binned,,
ttm <= 1 month,0.719377,0.719377
ttm (1-2] month,0.720619,0.720619
ttm (2-3] month,0.711341,0.711341
ttm (3-6] month,0.705610,0.705610
ttm (6-12] month,0.703654,0.703654
ttm > 12 month,0.683878,0.683878
